<a href="https://colab.research.google.com/github/udlbook/iclimbtrees/blob/main/notebooks/SAT_Crossword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crosswords with SAT

The purpose of this Python notebook is to use investigate using SAT to find a valid arrangement of known answers in a crossword puzzle.

You should have completed the notebook on SAT constructions before attempting this notebook.  Note:  this exercise is pretty hard.  Expect it to take a while!

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and write code to complete the functions.

You can save a local copy of this notebook in your Google account and work through it in Colab (recommended) or you can download the notebook and run it locally using Jupyter notebook or similar. If you are using CoLab, we recommend that turn off AI autocomplete (under cog icon in top-right corner), which will give you the answers and defeat the purpose of the exercise.

A fully working version of this notebook with the complete answers can be found [here](https://github.com/udlbook/iclimbtrees/blob/main/notebooks/SAT_Crossword_Answers.ipynb).

Contact me at iclimbtreesmail@gmail.com if you find any mistakes or have any suggestions.

In [ ]:
# Install relevant packages
!pip install z3-solver
from z3 import *
import numpy as np
import time

First let's write some code to visualize a crossword problem.  We'll represent the crossword as a ndarray of integers where each integer represents a letter index and zero represents a blank spot.  

In [ ]:
puzzle = ['ALCOVE NSEC MIC',
          'LEANED ALTO ADO',
          'LAVALAMPOON CON',
          'ASSN  EKG GABLE',
          '   DENTI MEMO  ',
          ' AEOLIANHARPOON',
          'MOANER SAX SKUA',
          'ERS MVP TWI PTS',
          'OTTO AUS ESPRIT',
          'WAILINGWALLOON ',
          '  NARA IDLES   ',
          'REDYE UMA  ECHO',
          'ARI FILMBUFFOON',
          'JOE UTNE SLOPPY',
          'ASS LEAR CORTEX']

# Convert to a list of lists
for i in range(len(puzzle)):
  puzzle[i] = [char for char in puzzle[i]]

# Represent the puzzle as integers in a grid
puzzle_as_integers = np.zeros((len(puzzle), len(puzzle[0])), dtype=int)
for i in range(len(puzzle)):
  for j in range(len(puzzle[i])):
    if puzzle[i][j] == ' ':
      puzzle_as_integers[i][j] = 0
    else:
      puzzle_as_integers[i][j] = ord(puzzle[i][j]) - ord('A') + 1

print(puzzle)
print(puzzle_as_integers)

Let's write a routine that draws this out nicely

In [ ]:
def draw_crossword(puzzle_as_integers):

  # Find number of rows and columns
  n_rows = puzzle_as_integers.shape[0]
  n_cols = puzzle_as_integers.shape[1]

  # Draw the top row
  print("╔", end="")
  for i in range(n_cols-1):
    print("═╤", end="")
  print("═╗")

  for c_row in range(n_rows):
    print("║", end="")
    for c_col in range(n_cols):
      if puzzle_as_integers[c_row][c_col] == 0:
        print(u"\u2588", end="")  # Use block character for blank spaces
      else:
        print(chr(puzzle_as_integers[c_row][c_col] + ord('A') - 1), end="")
      if(c_col < n_cols-1):
        print("│", end="")
    print("║")


  # Draw the bottom row
  print("╚", end="")
  for i in range(n_cols-1):
    print("═╧", end="")
  print("═╝")

draw_crossword(puzzle_as_integers)

The goal of this notebook will be to take a set of words and create a crossword layout like this in an $n \times n$ grid.  We'll start with just a small set of clues.  

In [ ]:
words = ['JANE','AUSTEN','PRIDE','NOVEL','DARCY','SENSE','EMMA','ESTATE','BENNET','BATH']

This routine takes the words, the grid size and various sets of constraints (which we'll develop one at a time).  It then runs the solver and  displays the crossword.

In [ ]:
def solve_crossword (words, grid_size, add_constraint_set1, add_constraint_set2=None, add_constraint_set3=None, add_constraint_set4=None ):

  # Fail if longest string length is not large enough to fit in grid
  longest_string_length = max(len(word) for word in words)
  if (longest_string_length > grid_size):
    print("Grid too small, no solution")
    return ;

  start_time = time.time()
  # Set up the SAT solver
  s = Solver()

  # This is a dictionary indexed by the word itself that contains the possible start
  # positions of that word, and whether the word is horizontal or vertical
  # The number of possible positions depend on the grid size as the word cannot exceed
  # grid.
  placement_vars = {word: [[[z3.Bool(f'{word}_{orientation}_{y},{x}')
                                for x in range(grid_size-len(word)+1 if orientation=='h' else grid_size )]
                                for y in range(grid_size-len(word)+1 if orientation=='v' else grid_size )]
                                for orientation in ['h', 'v']]
                                for word in words}

  # We will also define variables that indicate which letter is at which position
  # There are 27 possible characters (26 letters and a blank)
  # The variable x_i,j,k says that letter k is at position (i,j) in the grid
  letter_posns = [[[ z3.Bool("x_{%d,%d,%d}"%((i,j,k))) for k in range(0,27)] for j in range(0,grid_size) ] for i in range(0,grid_size) ]

  # Add the first set of constraints
  s = add_constraint_set1(s, placement_vars, letter_posns, words, grid_size)
  # Add the second set of constraints if present
  if add_constraint_set2 is not None:
    s = add_constraint_set2(s, placement_vars, letter_posns, words, grid_size)
  # Add the third set of constraints if present
  if add_constraint_set3 is not None:
    s = add_constraint_set3(s, placement_vars, letter_posns, words, grid_size)
  # Add the fourth set of constraints if present
  if add_constraint_set4 is not None:
    s = add_constraint_set4(s, placement_vars, letter_posns, words, grid_size)

  # Check if it's SAT (creates the model)
  sat_result = s.check()
  print(f"Executed in {time.time()-start_time:.4f} seconds")
  print(sat_result)

  # If it is then draw crossword, otherwise return
  if sat_result == z3.sat:
      result = s.model()
      # Retrieve the letter position variables in the solution as [0,1] values
      x_vals = np.array([[[int(bool(result[z3.Bool("x_{%d,%d,%d}" % (i, j, k))])) for k in range(0,27)] for j in range(0,grid_size) ] for i in range(0,grid_size) ] )

      # Find the position of the true value -- this is now a 2D grid with a 0 where there is a space and a value 1-26 representing a letter
      solution = np.argmax(x_vals, axis=2)
      # Draw the solution
      draw_crossword(solution)
  else:
      print("No solution")

Here's a couple of helpful routines that we can make use of

In [ ]:
# Takes a list of z3.Bool variables and returns constraints
# ensuring that there is exactly one true
def exactly_one(x):
  return PbEq([(i,1) for i in x],1)

# Converts a word in capital letters to its indices so 'ABD' becomes [1,2,4]
def letter_to_index(word):
  return [ord(char) - ord('A') + 1 for char in word]

Let's work on the first set of constraints.  


1.   Each word can only appear at one valid position
2.   Each position in the grid can have only a single letter present
3.   The letters implied by the word positions must agree where the words overlap



In [ ]:
def add_constraint_set1(s, placement_vars, letter_posns, words, grid_size):
  # Constraint 1: Each word can only be placed in exactly one position
  for word in words:
    # TODO implement this constraint
    # Replace these lines
    print(placement_vars[word]) # Will help you understand what to do!
    s.add(placement_vars[word][0][0][0])

  # Constraint 2: Each grid position can only have one letter present
  for i in range(0,grid_size):
    for j in range(0,grid_size):
      #TODO implement this constraint
      # Replace this line
      s.add(letter_posns[0][0][0])

  # Constraint 3: If a word is in a given position and orientation, the letters at the
  # appropriate grid positions must correspond (uses the routine letter_to_index() defined above)
  for word in words:
    for i in range(0,grid_size):
      # We'll do the horizontal words for you.  Read this code closely.
      for j in range(0,grid_size-len(word)+1):
        for letter_index in range(0,len(word)):
          s.add(Implies(placement_vars[word][0][i][j], letter_posns[i][j+letter_index][letter_to_index(word)[letter_index]]))
      # TODO define an equivalent constraint for the vertical positions
      # Replace this line
      s.add(letter_posns[0][0][0])

  return s

In [ ]:
# Let's test this routine so far
solve_crossword(words, 10, add_constraint_set1)

If you did this correctly, you should see that the words are all in there, but we don't have blank spaces where we should.  We need to add two further constraints to improve matters

1. Horizontal words must have a blank space or an edge to the left and right of their positions.  Vertical words must have a blank space or and edge above or below their positions.
2. Any position that is not part of a word should be blank



In [ ]:
def add_constraint_set2(s, placement_vars, letter_posns, words, grid_size):
  # Constraint 1:  Horizontal words must either start in the first column or have a 0 to their left
  #                Horizontal words must either finish in the last column of have a 0 to their right
  #                Vertical words must either start in the first row or have a 0 above them
  #                Vertical words must either end in the last row of have a 0 below them
  for word in words:
    # Horizontal words -- We'll do this one for you (read this code carefully)
    for i in range(grid_size):
        for j in range(1, grid_size - len(word)+1 ):
            # Check for border or blank square before the word starts
            s.add(Implies(placement_vars[word][0][i][j], letter_posns[i][j-1][0]))
            s.add(Implies(placement_vars[word][0][i][j-1], letter_posns[i][j+len(word)-1][0]))

    # Vertical words
    for i in range(1,grid_size - len(word)+1 ):
        for j in range(grid_size):
            # TODO -- write the equivalent constraint for the vertical words
            # Replace this line
            s.add(letter_posns[0][0][0])

  # Constraint 2:  Any position in the crossword grid that is not part of a word must be a blank space
  #                This stops random characters appearing outside the solution
  for i in range(grid_size):
      for j in range(grid_size):
          # Create a list of placement variables that add a letter to the current square
          relevant_placements = []
          for word in words:
              # Horizontal words
              for col in range(grid_size - len(word) + 1):
                  if j >= col and j < col + len(word):
                      relevant_placements.append(placement_vars[word][0][i][col])

              # Vertical words
              for row in range(grid_size - len(word) + 1):
                  if i >= row and i < row + len(word):
                      relevant_placements.append(placement_vars[word][1][row][j])


          # If none of the relevant placements are true, the square must be blank
          # TODO implement this constraint
          # Replace this line
          s.add(letter_posns[0][0][0])

  return s

In [ ]:
# Let's test this routine so far
solve_crossword(words, 10, add_constraint_set1, add_constraint_set2)

If you did this correctly, it should be an improvement, but it's not perfect; every letter is now part of either a horizontal or a vertical word.  However, when there are several vertical words adjacent to each other and we read horizontally across these words, we get nonsense. Similarly, when there are several horizontal words adjacent to each other an we read vertically through these words, we get nonsense.  We can fix this by adding another constraint:

*   If a letter is in a horizontal word, it is either inside a vertical word as well *OR* it has a blank square (or the edge of the grid) above and below it.
*   If a letter is in a vertical word, it is either inside a horizontal word as well *OR* it has a blank square (or the edge of the grid) to the left and the right of it.

This one is pretty tricky to get right, so just read the code and try to understand how it works.

In [ ]:
def add_constraint_set3(s, placement_vars, letter_posns, words, grid_size):
  # Constraint 1:   If a letter is in a horizontal word, it either
  #                     -- is inside a vertical word as well
  #                     -- has a blank (or edge) above and below it
  #                 If a letter in a vertical word exists, it is either
  #                     -- is inside a horizontal word too
  #                      -- has a blank (or edge) to the left and to the right of it.
  for i in range(0,grid_size):
      for j in range(0,grid_size):
          relevant_placements_horz = []
          relevant_placements_vert = []
          for word in words:
            for j2 in range (max(0,j-len(word)+1), min(j+1,grid_size-len(word)+1)):
                relevant_placements_horz.append(placement_vars[word][0][i][j2])
            for i2 in range(max(0,i-len(word)+1), min(i+1,grid_size-len(word)+1)):
                relevant_placements_vert.append(placement_vars[word][1][i2][j])
          in_horizontal_word = Or(relevant_placements_horz)
          in_vertical_word = Or(relevant_placements_vert)

          if(i == 0):
            above_and_below_are_blank = letter_posns[i+1][j][0]
          else:
            if(i == grid_size-1):
              above_and_below_are_blank = letter_posns[i-1][j][0]
            else:
              above_and_below_are_blank = And(letter_posns[i-1][j][0],letter_posns[i+1][j][0])

          if(j == 0):
            left_and_right_are_blank = letter_posns[i][j+1][0]
          else:
            if(j == grid_size-1):
              left_and_right_are_blank = letter_posns[i][j-1][0]
            else:
              left_and_right_are_blank = And(letter_posns[i][j-1][0],letter_posns[i][j+1][0])
          s.add(Implies(in_horizontal_word, Or(in_vertical_word, above_and_below_are_blank)))
          s.add(Implies(in_vertical_word, Or(in_horizontal_word, left_and_right_are_blank)))

  return s

In [ ]:
# Let's see how this improves things
solve_crossword(words, 10, add_constraint_set1, add_constraint_set2, add_constraint_set3)

If you've done this correctly, it should be working better, but we now have the problem that words do not all connect to each other.  Hence, we have to add a final constraint that all of the letters are connected.

First we form an $N\times N$ adjacency matrix which the value is true if word $i$ intersects with word $j$.  Then we use the 'is_fully_connected' construction that we developed in the notebook on SAT constructions.

In [ ]:
def is_fully_connected(s, adjacency):
  # Size of the adjacency matrix
  n_components = len(adjacency)
  # We'll construct a N x N x log[N] array of variables
  # The NxN variables in the first layer represent A, the variables in the second layer represent B and so on
  n_layers = math.ceil(math.log(n_components,2))+1
  connected = [[[ z3.Bool("conn_{%d,%d,%d}"%((i,j,n))) for n in range(0, n_layers)] for j in range(0, n_components) ] for i in range(0, n_components) ]

  # Constraint 1
  # The value in the top layer of the connected structure is equal to the adjacency matrix
  for i in range(n_components):
    for j in range(n_components):
      s.add(connected[i][j][0]==adjacency[i][j])

  # Constraint 2
  # Value at position [i,j] in layer n is value at position [i,j] of the binary matrix product of layer n-1 with itself
  for n in range(1,n_layers):
    for i in range(n_components):
      for j in range(n_components):
        matrix_entry_ij = False
        for k in range(n_components):
          matrix_entry_ij = Or(matrix_entry_ij, And(connected[i][k][n-1],connected[k][j][n-1]))
        s.add(connected[i][j][n]==matrix_entry_ij)

  # Constraint 3 -- any row of column of the matrix should be full of ones at the end (everything is connected)
  for i in range(n_components):
    s.add(connected[i][0][n_layers-1])

  return s

In [ ]:
# Helper routine that returns true if the current word is at position (i,j) in the grid
def word_at_position_ij(i,j, placement_vars, word, grid_size):

    relevant_placements = [] ;
    # Deal with horizontal words first
    for horz_pos in range(np.max([0, j-len(word)+1]), np.min([j+1, grid_size-len(word)+1])):
      # First the horizontal words
      relevant_placements.append(placement_vars[word][0][i][horz_pos])
    # Then the vertical words
    for vert_pos in range(np.max([0, i-len(word)+1]), np.min([i+1, grid_size-len(word)+1])):
      relevant_placements.append(placement_vars[word][1][vert_pos][j])

    return Or(relevant_placements) ;

In [ ]:
def add_constraint_set4(s, placement_vars, letter_posns, words, grid_size):
  # First lets create a new variable that represents the adjacency matrix of the words
  adjacency = [[ z3.Bool("adj_{%d,%d}"%((i,j))) for j in range(0, len(words)) ] for i in range(0, len(words)) ]

  # Run through each word
  for c_w1 in range(len(words)):
    for c_w2 in range(c_w1, len(words)):
      # If word indices are the same (i.e., c_w1=c_w2) then adjacency at c_w1,c_w2 is true
      # TODO -- replace this line
      s.add(adjacency[0][0])

      word1 = words[c_w1]
      word2 = words[c_w2]
      # TODO determine if word1 and word2 intersect.  You can use the routine "word_at_position_ij" above
      # Replace this line
      words_intersect = True

      # Set value and symmetric value of adjacency matrix
      s.add(adjacency[c_w1][c_w2] == words_intersect)
      s.add(adjacency[c_w2][c_w1] == adjacency[c_w1][c_w2])

  # Add the constraint that the adjacency matrix must be fully connected
  s = is_fully_connected(s, adjacency)
  return s

In [ ]:
# Let's see how this improves things (took 32 seconds to run for me, but might be longer)
solve_crossword(words, 11, add_constraint_set1, add_constraint_set2, add_constraint_set3, add_constraint_set4)

In [ ]:
# Now let's see what the smallest grid we can fit this in is.  The longest word is 6 letters, so can't be shorter than this
solve_crossword(words, 10, add_constraint_set1, add_constraint_set2, add_constraint_set3, add_constraint_set4)
solve_crossword(words, 9, add_constraint_set1, add_constraint_set2, add_constraint_set3, add_constraint_set4)
solve_crossword(words, 8, add_constraint_set1, add_constraint_set2, add_constraint_set3, add_constraint_set4)
solve_crossword(words, 7, add_constraint_set1, add_constraint_set2, add_constraint_set3, add_constraint_set4)
solve_crossword(words, 6, add_constraint_set1, add_constraint_set2, add_constraint_set3, add_constraint_set4)

These were some random words that I chose for an example.  Here are some real words from an NY Times mini puzzle.  You can see that it recovers the minimal possible solution correctly.

In [ ]:
words2 = ['GUM','TAB','ERA','END','IRA','MAP','TIMWALZ','ONE','ELI','COATS','POPHITS', \
          'GOT', 'PIE','BIZ','SPA','ALLSTAR','UNICORN','MEMOPAD','WAH','TEATIME']
solve_crossword(words2, 7, add_constraint_set1, add_constraint_set2, add_constraint_set3, add_constraint_set4)

Further work for keeners:

1. Technically, we should also put in a constraint to stop two words starting in the same place.  For example, the current solver would allow the words 'EAT' and 'EATEN' to start in the same place and overlap.  Add this constraint.

2. As proposed, this is fairly impractical to help make crosswords.  It would be more useful to have a superset of answers and allow the SAT solver to choose from these to make a sensible crossword.  You would then ask the SAT solver to build an crossword of size $N$ that included at least $W$ words.  Add this feature.
